In [1]:
cd /content/drive/My Drive/OD segmentation/

/content/drive/My Drive/OD segmentation


In [2]:
import numpy as np
#import scipy.io as sio
import scipy.misc
from keras.preprocessing import image
from skimage.transform import rotate, resize
from skimage.measure import label, regionprops
from time import time
from utils_Mnet import pro_process, BW_img, disc_crop
import matplotlib.pyplot as plt
from skimage.io import imsave


import cv2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import Model_DiscSeg as DiscModel
import Model_MNet as MNetModel

Using TensorFlow backend.


In [11]:
!pip install keras==2.0.0
!pip install scipy==1.1.0

     |████████████████████████████████| 194kB 2.9MB/s 
  Created wheel for keras: filename=Keras-2.0.0-cp36-none-any.whl size=227983 sha256=f773f0d6d79e5780c4f801572da3bfd56e87877d7d1fee92b83f9a2b72e7df80
  Stored in directory: /root/.cache/pip/wheels/b0/a1/f4/f5f6a53dd71728cae0176b860f8c4304dd5d3fc2b8387ef59c
Successfully built keras
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.0 which is incompatible.
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


     |████████████████████████████████| 31.2MB 145kB/s 
ERROR: umap-learn 0.4.4 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [3]:
DiscROI_size = 400
DiscSeg_size = 640
CDRSeg_size = 400

train_data_type = '.jpg'
mask_data_type ='.bmp'

Original_validation_img_path = 'data/'

valiImage_save_path='data/save_path_460/'

PolarTrainImage_save_path='data/PolarTrainImage_save_path/'
seg_result_save_path = 'data/result_460/'

if not os.path.exists(seg_result_save_path):
    os.makedirs(seg_result_save_path)
    

if not os.path.exists(valiImage_save_path):
    os.makedirs(valiImage_save_path)        


if not os.path.exists(PolarTrainImage_save_path):
    os.makedirs(PolarTrainImage_save_path)

In [4]:
file_train_list = [file for file in os.listdir(Original_validation_img_path) if file.lower().endswith(train_data_type)]

DiscSeg_model = DiscModel.DeepModel(size_set=DiscSeg_size)
DiscSeg_model.load_weights('Model_DiscSeg_ORIGA_pretrain.h5')

CDRSeg_model = MNetModel.DeepModel(size_set=CDRSeg_size)
CDRSeg_model.load_weights('Model_MNet_ORIGA_pretrain.h5')

AttributeError: ignored

In [14]:

for lineIdx in range(0, len(file_train_list)):
    temp_txt = [elt.strip() for elt in file_train_list[lineIdx].split(',')]

    org_img = np.asarray(image.load_img(Original_validation_img_path + temp_txt[0]))
    plt.imshow(org_img)
    plt.title('org_img')
    plt.show()

    
    # Disc region detection by U-Net
    temp_org_img = resize(org_img, (DiscSeg_size, DiscSeg_size, 3)) 

 
    temp_org_img = np.reshape(temp_org_img, (1,) + temp_org_img.shape)*255

    [prob_6, prob_7, prob_8, prob_9, prob_10] = DiscSeg_model.predict([temp_org_img])
    
    OD = np.squeeze(np.clip(prob_10*255,0,255).astype('uint8'));
    plt.imshow(OD)
    plt.title('temp_img')
    plt.show()
    

    
    OD=OD /OD.max()
    OD=255*OD
    OD=OD.astype(np.uint8)
    OD[OD==255]=200
    OD[OD==0]=0   
    OD[OD==200]=255   
    OD[(OD<200) & (OD>0)]=128  
    imsave(seg_result_save_path+temp_txt[0],(OD))
    
    